# Sub Interfaces over Dedicated Network Interface Cards

FABRIC allows users to utilize dedicated NIC cards `NIC_ConnectX_5` and `NIC_ConnectX_6`. This notebook outlines the process for creating sub-interfaces for these NICs and connecting them to Network Services. Specifically, we will demonstrate how to connect the sub-interfaces to the FABRIC FABnetv4 Network Service. However, the sub-interfaces can be connected to any of the services listed in the Network Section of [Start Here](../../../start_here.ipynb).



## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

In this example, we will create a slice and configure nodes with dedicated NICs, sub-interfaces, and network connections.

NIC component models options:
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

### Define variables

In [ ]:
slice_name = 'MySlice'

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'
network3_name='net3'

model = "NIC_ConnectX_6"

### Identify the Sites

List available sites with `NIC_ConnectX_5` or `NIC_ConnectX_6`. Chooose two site at random from the available sites.

In [ ]:
# we will use CX5 to generate traffic and CX6 to mirror traffic into so need sites that have both for this example.
cx5_column_name = 'nic_connectx_5_available'
cx6_column_name = 'nic_connectx_6_available'

# find two sites with available ConnectX-5 and ConnectX-6 cards
(site1, site2) = fablib.get_random_sites(count=2, filter_function=lambda x: x[cx6_column_name] > 0)

print(f"Sites chosen: {site1} {site2}")

### Slice Creation
First, we create a new slice.

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

### Network Setup
We then define three networks:

- `net1` and `net2` are Layer 3 networks using IPv4.
- `net3` is a Layer 2 network.

In [ ]:
# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')
net3 = slice.add_l2network(name=network3_name, subnet="192.168.1.0/24")

### Node1 Configuration
We add `Node1` to the slice and configure it as follows:

- Add a dedicated NIC (`NIC_ConnectX_6`).
- Create two sub-interfaces on the NIC.
- Connect one sub-interface to `net1` and set up routing to FABnetv4.
- Connect the other sub-interface to `net3` for a Layer 2 Wide Area connection.

In [ ]:
node1 = slice.add_node(name=node1_name, site=site1)
node1_iface1 = node1.add_component(model=model, name='nic1').get_interfaces()[0]

node1_ch_iface1 = node1_iface1.add_sub_interface("child1", vlan="100")
node1_ch_iface1.set_mode('auto')
net1.add_interface(node1_ch_iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())


node1_ch_iface2 = node1_iface1.add_sub_interface("child2", vlan="200")
node1_ch_iface2.set_mode('auto')
net3.add_interface(node1_ch_iface2)

### Node2 Configuration
We add `Node2` to the slice and configure it as follows:

- Add a dedicated NIC (`NIC_ConnectX_6`).
- Create two sub-interfaces on the NIC.
- Connect one sub-interface to `net2` and set up routing to FABnetv4.
- Connect the other sub-interface to `net3` for a Layer 2 Wide Area connection.

In [ ]:
node2 = slice.add_node(name=node2_name, site=site2)
node2_iface1 = node2.add_component(model=model, name='nic1').get_interfaces()[0]

node2_ch_iface1 = node2_iface1.add_sub_interface("child1", vlan="100")
node2_ch_iface1.set_mode('auto')
net2.add_interface(node2_ch_iface1)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())

node2_ch_iface2 = node2_iface1.add_sub_interface("child2", vlan="200")
node2_ch_iface2.set_mode('auto')
net3.add_interface(node2_ch_iface2)

### Submit Slice Request
Finally, we submit the slice request to instantiate the configuration.

In [ ]:
slice.submit()

## Run the Experiment

Verify connectivity through the sub-interfaces for two different network services:
- Ensure the network passes over FabNetV4.
- Ensure the network passes over the Layer 2 Wide Area Network.



In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network3_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()